In [ ]:
!pip install mlxtend

  Using cached mlxtend-0.22.0-py2.py3-none-any.whl (1.4 MB)


In [ ]:
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.parse import urlparse
import yaml
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [ ]:
Data = pd.read_csv('C:/Users/ASUS/Desktop/DataSet/DataSet.csv', header=0, index_col=0)

In [ ]:
Data.shape

(50315, 12741)

In [ ]:
counts = Data.sum()
# Créer un nouveau dataframe pour le tableau
Columns_sum = pd.DataFrame({"Nom de l'action": Data.columns, 'Nombre d\'utilisations': counts})
Columns_sum = Columns_sum.sort_values('Nombre d\'utilisations', ascending=False)

In [ ]:
Columns_sum

,Nom de l'action,Nombre d'utilisations
actions/checkout@v2,actions/checkout@v2,28284.0
actions/checkout@v3,actions/checkout@v3,16969.0
actions/setup-node@v1,actions/setup-node@v1,5373.0
actions/checkout@v1,actions/checkout@v1,4202.0
actions/setup-node@v3,actions/setup-node@v3,4166.0
...,...,...
docker://pointnetwork/pointnetwork_node:v0.1.83,docker://pointnetwork/pointnetwork_node:v0.1.83,1.0
docker://pipelinecomponents/remark-lint:latest,docker://pipelinecomponents/remark-lint:latest,1.0
docker://phparkitect/arkitect-github-actions:latest,docker://phparkitect/arkitect-github-actions:l...,1.0
docker://peaceiris/gh-pages:v2,docker://peaceiris/gh-pages:v2,1.0


In [ ]:
Columns_sum.describe()

,Nombre d'utilisations
count,12741.000000
mean,15.858331
std,316.953982
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,28284.000000


***Extraction des projets***

In [ ]:
projets = Data.index.tolist()

In [ ]:
len(projets)

50315

In [ ]:
def extract_github_info(url):
    language = []
    # Faire la requête HTTP pour obtenir le contenu HTML de la page
    r = requests.get(url)

    # Vérifier si la requête a réussi
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'html.parser')
        parsed_url = urlparse(url)
        repository_name = parsed_url.path.strip('/')
        stars_elem = soup.find('a', href="/{}/stargazers".format(repository_name))
        if stars_elem:
            stars = stars_elem.find('span', class_='text-bold').text.strip()
        else:
            stars = 0
        forks_elem = soup.find('a', href="/{}/forks".format(repository_name))
        if forks_elem:
            forks = forks_elem.find('span', class_='text-bold').text.strip()
        else:
            forks = 0
        branches_elem = soup.find_all('a', href="/{}/branches".format(repository_name))
        if branches_elem:
            branches = branches_elem[1].find('strong').text.strip()
        else:
            branches= 0
        default_branche_elem = soup.find('span', class_='css-truncate-target', attrs={'data-menu-button': True})
        if default_branche_elem:
            default_branche= default_branche_elem.text.strip()
        else:
            default_branche = ""
        commits_elem = soup.find('a', href=f"/{repository_name}/commits/{default_branche}")
        if commits_elem:
            commits = commits_elem.find('strong').text.strip()
        else:
            commits = 0
        contributors_elem = soup.find('a', href='/{}/graphs/contributors'.format(repository_name))
        #contributors = contributors_elem.find('span').text.strip()
        if contributors_elem:
            contributors_1 = contributors_elem.find('span')
            if contributors_1:
                contributors= contributors_1.text.strip()
            else:
                contributors = None
        else:
            contributors = None
        languages_div = soup.find_all("ul", class_="list-style-none")
        if len(languages_div)>2:
            language_items = languages_div[-2].find_all("li", class_="d-inline")
            for item in language_items:
                language_name = item.find("span", class_="color-fg-default text-bold mr-1").text.strip()
                language.append(language_name)
        Issues_elem = soup.find('a', href="/{}/issues".format(repository_name))
        if Issues_elem:
            issues = Issues_elem.find("span", id="issues-repo-tab-count").text.strip()
        else:
            issues = 0

        Pulls_elem = soup.find('a', href="/{}/pulls".format(repository_name))
        if Pulls_elem:
            pulls = Pulls_elem.find("span", id="pull-requests-repo-tab-count").text.strip()
        else:
            pulls = 0
        description = soup.find("meta", attrs={"name": "description"})["content"].split('- GitHub -')[0]


        # Créer un dictionnaire pour stocker les informations extraites
        project_info = {
            'URL': url,
            'Name' : repository_name,
            'Forks': forks,
            'Stars': stars,
            'Branches': branches,
            'Default_Branches' : default_branche,
            'Commits' : commits,
            'Contributors' : contributors,
            'Languages': language,
            'Issues': issues,
            'Pulls': pulls,
            'Description' : description,

        }

        return project_info
    else:
        # Si la requête a échoué, retourner None
        return None


In [ ]:
# Liste pour stocker les DataFrames extraites de chaque URL
github_info_dfs = []
# Parcourir les URLs et appeler la fonction extract_github_info pour chaque URL
for url in projets:
    github_info_df = extract_github_info(url)
    if github_info_df is not None:
        github_info_dfs.append(github_info_df)


In [ ]:
# Concaténer les DataFrames en une seule DataFrame globale
data_projects = pd.DataFrame(github_info_dfs).set_index('URL')

In [ ]:
data_projects.shape

(50315, 12)

In [ ]:
data_projects.isna().sum()

URL                     0
Name                    0
Forks                   0
Stars                   0
Branches                0
Default_Branches        1
Commits                 0
Contributors        15492
Languages               0
Issues                  0
Pulls                   0
Description             0
dtype: int64

In [ ]:
# Fonction pour convertir les valeurs en entier
def convert_to_int(value):
    if 'k' in str(value):
        return int(float(value[:-1]) * 1000)
    elif pd.isna(value):
        return 0
    else:
        return int(value)

In [ ]:
data_projects['Commits']=data_projects['Commits'].apply(convert_to_int)
data_projects['Stars']=data_projects['Stars'].apply(convert_to_int)
data_projects['Forks']=data_projects['Forks'].apply(convert_to_int)
data_projects['Branches']=data_projects['Branches'].apply(convert_to_int)
data_projects['Issues']=data_projects['Issues'].apply(convert_to_int)
data_projects['Pulls']=data_projects['Pulls'].apply(convert_to_int)
data_projects['Contributors']=data_projects['Contributors'].apply(convert_to_int)

In [ ]:
data_projects =data_projects.set_index('URL')

In [ ]:
data_projects.to_csv('C:/Users/ASUS/Desktop/DataSet/Projects.csv')